In [1]:
import sys

import pandas as pd
from pandas import DataFrame, Series

import numpy as np

import matplotlib.pyplot as plt
import matplotlib

#import xgboost as xgb

from sklearn import tree
from sklearn.ensemble import RandomForestClassifier 

%matplotlib inline

In [3]:
train = pd.read_csv("../data/train.csv", index_col=0)
test = pd.read_csv("../data/test.csv", index_col=0)

def categorising(dataset):
    opt = dataset.copy()

    opt["Intact"] = opt.SexuponOutcome.replace("Intact Male", "Yes")
    opt.Intact = opt.Intact.replace("Neutered Male", "No")
    opt.Intact = opt.Intact.replace("Spayed Female", "No")
    opt.Intact = opt.Intact.replace("Intact Female", "Yes")
    opt.Intact = opt.Intact.replace("Unknown", np.nan)

    opt.Intact = opt.Intact.astype("category")
    opt["IntactValue"] = opt.Intact.cat.codes

    opt.SexuponOutcome = opt.SexuponOutcome.replace("Neutered Male", "Male")
    opt.SexuponOutcome = opt.SexuponOutcome.replace("Intact Male", "Male")
    opt.SexuponOutcome = opt.SexuponOutcome.replace("Spayed Female", "Female")
    opt.SexuponOutcome = opt.SexuponOutcome.replace("Intact Female", "Female")
    opt.SexuponOutcome = opt.SexuponOutcome.replace("Unknown", np.nan)

    opt.SexuponOutcome = opt.SexuponOutcome.astype("category")
    opt["SexuponOutcomeValue"] = opt.SexuponOutcome.cat.codes

    return opt

train = categorising(train)
test = categorising(test)

opt = train.copy()

opt.Intact.value_counts()

No     18599
Yes     7036
Name: Intact, dtype: int64

In [4]:
factor = {'year': 365, 'month': 31, 'week': 7, 'day': 1}
result = []
for age in opt.AgeuponOutcome:
    if str(age) == 'nan':
        result.append(np.nan)
        continue
    value, unit = age.split(' ')
    days = int(value) * factor[unit.replace('s', '')] # ignore year[s], month[s], ...
    result.append(days)
opt.AgeuponOutcomeInDays = pd.Series(result)
assert len(opt.AgeuponOutcomeInDays) == len(opt.AgeuponOutcome)

In [6]:
result = []
age_categories = [['younger than 1 years', 1*365.], 
                  ['younger than 2 years', 2*365.], 
                  ['younger than 5 years', 5*365.],
                  ['older than 5 years', 990*365.]]
for age in opt.AgeuponOutcomeInDays:
    if str(age) == 'nan':
        result.append(np.nan)
        continue
    for limit in age_categories:
        if age < limit[1]:
            result.append(limit[0])
            break
    else:
        print(age)
opt.AgeuponOutcomeCategory = pd.Series(result)
opt.AgeuponOutcomeCategory = opt.AgeuponOutcomeCategory.astype("category")
assert len(opt.AgeuponOutcomeCategory) == len(opt.AgeuponOutcome)
opt.AgeuponOutcomeCategory.value_counts()

younger than 1 years    11890
younger than 5 years     6636
older than 5 years       4216
younger than 2 years     3969
dtype: int64